In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/Project DeepLearning/

/content/drive/MyDrive/Project DeepLearning


In [ ]:
!pip install vncorenlp

     |████████████████████████████████| 2.6 MB 7.9 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=f054224449ef83b3c558c932e8a8b5f7fb0ceb059a88051e4eb1d810e172be6f
  Stored in directory: /root/.cache/pip/wheels/0c/d8/f2/d28d97379b4f6479bf51247c8dfd57fa00932fa7a74b6aab29
Successfully built vncorenlp


IMPORT THƯ VIỆN

In [ ]:
import gensim
import xlrd
import numpy as np
import matplotlib.pyplot as plt

from vncorenlp import VnCoreNLP
import pandas as pd
import pickle
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix,ConfusionMatrixDisplay
from sklearn import metrics

from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import *

Thiết lập GPU

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
tf.device('/device:GPU:0')

Found GPU at: /device:GPU:0


# Xử lý dữ liệu văn bản thành dạng vector với Word2Vec

LOAD MODEL WORD2VEC

In [ ]:
# model_baomoi= baomoi.vn.model.bin
model_name = "Word2Vec/baomoi.vn.model.bin"
w2v = gensim.models.KeyedVectors.load_word2vec_format(model_name, binary= True)
w2v_weights = w2v.wv.vectors
vocab_size, embedding_size = w2v_weights.shape

print(w2v.similar_by_word('việt_nam', topn=20))
print(w2v_weights.shape)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


[('viêt_nam', 0.7453469038009644), ('vn', 0.696385383605957), ('nhật_bản', 0.6912186145782471), ('hàn_quốc', 0.6518356800079346), ('việtnam', 0.6430970430374146), ('trung_quốc', 0.6374214887619019), ('myanmar', 0.6262404322624207), ('indonesia', 0.610821545124054), ('hoa_kỳ', 0.6042898297309875), ('singapore', 0.6018333435058594), ('thái_lan', 0.5947649478912354), ('Ấn_Độ', 0.5873130559921265), ('australia', 0.568601131439209), ('philippines', 0.5652974843978882), ('mỹ', 0.5640307664871216), ('cuba', 0.5515502691268921), ('Đài_loan', 0.5479223728179932), ('malaysia', 0.5464203357696533), ('campuchia', 0.5429700016975403), ('hoa_kì', 0.533842921257019)]
(439056, 300)


In [ ]:
def seq2tokens(seq):
    tokens = []
    for word in seq:
        # word = word.replace('_',u'\xa0')
        if (word in w2v.wv.vocab):
            tokens.append(w2v.wv.vocab[word].index)
    return tokens

def token2seq(tokens):
    return [w2v.wv.index2word[token] for token in tokens]

LOAD DỮ LIỆU


In [ ]:
MAX_LENGTH = 500
number_topic = 12
rdrsegmenter = VnCoreNLP("vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

def get_data(raw_text):
    x = []
    temp_x = raw_text
    for sample in temp_x :     
        sample = rdrsegmenter.tokenize(sample)  
        sample = ' '.join([' '.join(sent) for sent in sample])
        sample = gensim.utils.simple_preprocess(sample)
        x.append(seq2tokens(sample))

    x = pad_sequences(x,maxlen=MAX_LENGTH).tolist()
    return x

Load dữ liệu

In [ ]:
data_train = pickle.load(open('Data/data_train.pkl', 'rb'))
data_test = pickle.load(open('Data/data_test.pkl', 'rb'))
target_train = pickle.load(open('Data/target_train.pkl', 'rb'))
target_test = pickle.load(open('Data/target_test.pkl', 'rb'))

In [ ]:
x_train= get_data(data_train)
x_test= get_data(data_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [ ]:
y_train = pd.get_dummies(target_train).values.tolist()
y_test = pd.get_dummies(target_test).values.tolist()

In [ ]:
pickle.dump(x_train, open('Word2Vec/x_train.pkl', 'wb'))
pickle.dump(y_train, open('Word2Vec/y_train.pkl', 'wb'))

pickle.dump(x_test, open('Word2Vec/x_test.pkl', 'wb'))
pickle.dump(y_test, open('Word2Vec/y_test.pkl', 'wb'))

Có sẵn thì không cần xử lý

In [ ]:
MAX_LENGTH = 500
x_train = pickle.load(open('Word2Vec/x_train.pkl', 'rb'))
y_train = pickle.load(open('Word2Vec/y_train.pkl', 'rb'))

x_test = pickle.load(open('Word2Vec/x_test.pkl', 'rb'))
y_test = pickle.load(open('Word2Vec/y_test.pkl', 'rb'))

# Thống kê phân loại

In [ ]:
id =[]
for i in range(number_topic):
   id.append(np.where( np.argmax(y_train, axis = 1) == i)[0])
   np.random.shuffle(id[i])

print([len(id[i]) for i in range(number_topic)])

[2006, 1836, 1927, 1978, 1856, 1890, 1737, 1942, 1802, 2388, 2033, 1909]


# TRAINING


Chia tập train, validate

In [ ]:
X_train, X_val, Y_train, Y_val = sklearn.model_selection.train_test_split(np.array(x_train), np.array(y_train), test_size=0.1, random_state=42)
x_test = np.array(x_test)
y_test = np.array(y_test)

Đánh giá kết quả và visualize

In [ ]:
def visual_history_train(history):
    # list all data in history
    key = history.history.keys()
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
def evaluate(classifier, X_val, Y_val, X_test, Y_test):
    val_predictions = classifier.predict(X_val).argmax(axis=-1)
    test_predictions = classifier.predict(X_test).argmax(axis=-1)
    cm = confusion_matrix(Y_test.argmax(axis=-1), test_predictions)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.show()
    print("Validation accuracy: ", metrics.accuracy_score(val_predictions, Y_val.argmax(axis=-1)))
    print("Test accuracy: ", metrics.accuracy_score(test_predictions, Y_test.argmax(axis=-1)))

LSTM MODEL


In [ ]:
def create_LSTM_model():
    classifier = Sequential()
    classifier.add(Embedding(input_dim = vocab_size,
                    output_dim = embedding_size,
                    weights=[w2v_weights],
                    input_length=MAX_LENGTH,
                    mask_zero=True,
                    trainable=False))  
    classifier.add(LSTM(embedding_size))
    
    classifier.add(Dense(number_topic,activation="softmax"))

    classifier.summary()
    classifier.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])

    return classifier

LSTM_model = create_LSTM_model()
LSTM_history = LSTM_model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=10, batch_size= 64)
# train_model(classifier = LSTM_model, X_data=x_train, Y_data=y_train, X_test=x_test, Y_test=y_test,n_epochs=2)

In [ ]:
# print("Test accuracy: ", LSTM_model.evaluate(x_test,y_test)[1])
evaluate(LSTM_model, X_val=X_val, Y_val = Y_val,X_test= x_test, Y_test=y_test)
visual_history_train(LSTM_history)
LSTM_model.save("Word2Vec/model/LSTM_model.save")

BiLSSTM Model

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.01)


def create_BiLSTM_model():
    model = Sequential()
    model.add(Embedding(input_dim = vocab_size,
                    output_dim = embedding_size,
                    weights=[w2v_weights],
                    input_length=MAX_LENGTH,
                    mask_zero=True,
                    trainable=False))

    model.add(Bidirectional(LSTM(embedding_size)))
    model.add(Dense(256,activation="relu"))
    model.add(Dense(number_topic,activation="softmax"))
    
    model.summary()
    model.compile(optimizer=opt,loss="categorical_crossentropy",metrics=['accuracy'])
    return model

BiLSTM_model = create_BiLSTM_model()
BiLSTM_history = BiLSTM_model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=10, batch_size= 64)

In [ ]:
evaluate(BiLSTM_model, X_val=X_val, Y_val = Y_val,X_test= x_test, Y_test=y_test)
visual_history_train(BiLSTM_history)
BiLSTM_model.save("Word2Vec/model/BiLSTM_model.save")